# Функции: избранные темы

## Вступление: контекстные менеджеры

В Python по умолчанию задана некоторая точность вычисления чисел с плавающей точкой, которая, что логично, может отличаться от требуемой для решения конкретной задачи точности. Для пример, будет рассматривать точность вычислений в библиотеке **decimal**.

Предположим, нас устраивает точность вычислений по умолчанию в этой библиотеке, но для некоторых расчетов мы бы хотели использовать повышенную или пониженную точность и только для них. Как нам быть? Тут-то нам на помощь и придут контекстные менеджеры.

В этом примере мы поговорим о контекстных менеджерах и напишем свой менеджер для установки точности вычислений, которые будет работать примерно так:

```python
with set_precision(5):
    ...
```

In [10]:
from decimal import Decimal, getcontext
from contextlib import contextmanager
from typing import Generator


@contextmanager
def set_precision(precision: int) -> Generator[None, None, None]:
    precision = int(precision)

    if precision < 1:
        raise ValueError("precision should be non negative integer")

    #сохранение контекста
    precision_old = getcontext().prec
    getcontext().prec = precision

    #возвращение управления
    try:
        yield

    #восстановление контекста
    finally:
        getcontext().prec = precision_old
        print('restire precision')

In [11]:

num1, num2 = Decimal('3.14'), Decimal('2.72')
print(num1 / num2)

print(getcontext().prec)

with set_precision(128):
    print(num1 / num2)
    raise Exception



1.154411764705882352941176471
28
1.1544117647058823529411764705882352941176470588235294117647058823529411764705882352941176470588235294117647058823529411764705882
restire precision


Exception: 

## Разминка: геометрическая прогрессия

Напишите бесконечный генератор геометрической прогрессии. В качестве параметров генератор должен принимать:  
- первый член прогрессии
- шаг прогрессии

In [12]:
from typing import Generator


def generate_geometry_progression(
    start: float = 1,
    step: float = 1
) -> Generator[float, None, None]:
    start, step = float(start), float(step)

    current_member = start

    while True:
        yield current_member
        current_member *= step

In [13]:
generator = generate_geometry_progression(step=0.5)

for i in range(5):
    print(next(generator))

1.0
0.5
0.25
0.125
0.0625


## Задание 1: float_range

Необходимо создать аналог range(), который генерирует арифметическую прогрессию. Ниже приведены примеры использования:

*Пример 1*:

```python
for i in float_range(stop=5):
    print(i)
```

*Вывод*:
```console
0
1
2
3
4
```
___
*Пример 2*:

```python
for i in float_range(stop=1, step=0.5):
    print(i)
```

*Вывод*:
```console
0
0.5
```
___
*Пример 3*:

```python
for i in float_range(start=1, stop=-1, step=-0.5):
    print(i)
```

*Вывод*:
```console
1
0.5
0
-0.5
```

In [27]:
def float_range(start=0, stop=None, step=1) -> Generator[float, None, None]:
    if stop is None:
        raise ValueError("stop is not given")

    if any([type(start) not in (int, float), type(stop) not in (int, float), type(step) not in (int, float)]):
        raise ValueError("invalid value is given")

    if step == 0:
        raise ValueError("step can't be 0")
    elif stop < start and step > 0:
        raise ValueError("stop is less than start and step is non negative")
    elif stop > start and step < 0:
        raise ValueError("stop is more than start and step is non positive")

    current_member = start

    while True:
        yield current_member
        current_member += step

        if abs(current_member) >= abs(stop):
            break

In [28]:
for i in float_range(stop=5):
    print(i)
print("\n")

for i in float_range(stop=1, step=0.5):
    print(i)
print("\n")

for i in float_range(start=1, stop=-1, step=-0.5):
    print(i)
print("\n")

0
1
2
3
4


0
0.5


1
0.5
0.0
-0.5




## Задание 2: свой map

### Часть 1: копируем map

Реализуйте аналог функции map, полностью копирующий ее поведение. Саму map использовать нельзя.

In [22]:
def custom_map(func, *arguments):
    for args in zip(*arguments):
        yield func(*args)

In [27]:
l1 = [1,2,3,4,5,6,7]
print(list(custom_map(lambda x: x*2, l1)))

print(list(map(lambda x: x*2, l1)))

[2, 4, 6, 8, 10, 12, 14]


TypeError: <lambda>() takes 1 positional argument but 2 were given

### Часть 2: дополняем map

Добавьте возможность управлять поведением вашего map'a: сделайте так, чтобы map имела возможность не только обрезать последовательности, но и дополнять короткие последовательности до динных. 

Совет: функция **zip_longest** из библиотеки **itertools** может оказаться полезной.

In [28]:
from enum import Enum
from itertools import zip_longest


class MapTypes(Enum):
    SHORTEST = 'short'
    LONGEST = 'long'


In [34]:
def custom_map2(func, *arguments, type=MapTypes.SHORTEST, fill_value):
    if type is MapTypes.SHORTEST:
        for args in zip(*arguments):
            yield func(*args)
    else:
        for args in zip_longest(*arguments, fillvalue=fill_value):
            yield func(*args)

In [36]:
l1 = [1,2,3,4,5,6,7]
l2 = [34,23,5,34]
print(list(custom_map2(lambda x, y: (x+y)*2, l1, l2, type=MapTypes.LONGEST, fill_value=100)))

[70, 50, 16, 76, 210, 212, 214]


## Задание 3: Спиннер

### Часть 1: генератор

Напишите функцию, которая на вход получает коллекцию и возвращает генератор, последовательно возвращающий элементы коллекции, а после возврата последнего элемента коллекции очередной вызов генератора приведет к зацикливанию.


*Пример*:

```python
generator = generate_circle('abc')

print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
print(next(generator))
```
*Вывод*:
```console
a
b
c
a
b
```

In [1]:
from typing import Generator

def generate_circle(sequence) -> Generator:
    if sequence in (int, float):
        sequence = str(sequence)

    current_symbol = sequence[0]
    sequence_len = len(sequence)
    i = 1

    while True:
        yield current_symbol
        current_symbol = sequence[i]
        i = (i + 1) % sequence_len

In [4]:
generator = generate_circle()

for _ in range(10):
    print(next(generator))

1
2
4
4
2465
21
1
2
4
4


### Часть 2: Колесо Сансары

Используя генератор из предыдущего раздела, реализуйте функцию, которая отображает на экране спиннер для индикации загрузки.

Отрисовка спиннера печатает на экран надпись: Thinking: \<symbol\>, где вместо \<symbol\> последовательно появляются знаки: \, |, /, -, что создаёт эффект вращения.

Вход функции: 
- time_limit - время (в секундах), в течение которого должна производиться отрисовка спиннера;
- pause - время (в секундах) задержки между сменой символов спиннера;

Интересная статья на тему индикаторов: https://dtf.ru/flood/174240-progress-bar-ili-spinner-chto-i-kogda-ispolzovat?ysclid=lorrg51syv550654720

In [65]:
import time
from typing import Generator

def Sansara_circle(time_limit=2, pause=0.1) -> Generator:
    # if not(time_limit in (int, float) and pause in (int, float)):
    #     raise ValueError("wrong input")

    number = int(time_limit / pause)
    sequence = "|/-\\"
    seq_len = len(sequence)

    for i in range(number):
        time.sleep(pause)
        print(f"Thinking: {sequence[i % seq_len]}\r", end="")

In [67]:
Sansara_circle(5, 0.3)